# **Dados das Previsões do Modelo XG Boost**

Esse notebook tem como objetivo gerar as previsões dos preços de fechamento das ações, utilizando o modelo de regressão criado no notebook "previsao_acoes_ml".

* **O que será feito?**

    * Criação de uma base de dados aleatória 📊

        * São gerados dados sintéticos para os próximos 90 dias, respeitando faixas realistas de preços, volume de negociações e indicadores financeiros.

    * Carregamento do modelo e a geração das previsões 🤖

        * O modelo de regressão desenvolvido é carregado do arquivo modelo_xgboost.pkl

        * O modelo faz previsões dos preços de fechamento e salva os resultados no arquivo `previsoes_nova_base.csv` 

    * Importação da base de dados real
        * O conjunto de dados é `uber_stock_data.csv` é carregado para validar a estrutura dos dados históricos das ações.

        * Isso permite comparar futuras previsões com dados reais.

## 1. Criação de uma base de dados aleatória

In [1]:
import pandas as pd
import numpy as np

# Definir número de dias e datas
dias_futuros = 90
datas_futuras = pd.date_range(start="2025-01-01", periods=dias_futuros, freq="D")

# Gerar valores aleatórios dentro de faixas típicas para cada coluna
np.random.seed(42)  # Para reprodutibilidade dos dados
df_aleatorio = pd.DataFrame({
    "Data_Negociacao": datas_futuras,
    "Preco_Abertura": np.random.uniform(20, 60, dias_futuros),  # Valores entre $20 e $60
    "Maior_Preco_Dia": np.random.uniform(25, 65, dias_futuros),
    "Menor_Preco_Dia": np.random.uniform(15, 55, dias_futuros),
    "Volume_Negociacoes_Proc": np.random.uniform(10, 50, dias_futuros),
    "Media_Movel_7": np.random.uniform(20, 60, dias_futuros),
    "Media_Movel_30": np.random.uniform(20, 60, dias_futuros),
    "Media_Movel_90": np.random.uniform(20, 60, dias_futuros),
    "Volatilidade_7": np.random.uniform(0.5, 5, dias_futuros),  # Volatilidade mais baixa
    "Retorno_Percentual": np.random.uniform(-0.05, 0.05, dias_futuros)  # Retorno entre -5% e 5%
})

# Salvar a base em um arquivo CSV
df_aleatorio.to_csv("base_dados_aleatoria.csv", index=False)

# Exibir amostra dos dados gerados
print(df_aleatorio.head())


  Data_Negociacao  Preco_Abertura  Maior_Preco_Dia  Menor_Preco_Dia  \
0      2025-01-01       34.981605        29.783770        28.642654   
1      2025-01-02       58.028572        53.529791        19.538941   
2      2025-01-03       49.279758        55.431402        51.987745   
3      2025-01-04       43.946339        47.451088        50.093574   
4      2025-01-05       26.240746        55.838687        25.317665   

   Volume_Negociacoes_Proc  Media_Movel_7  Media_Movel_30  Media_Movel_90  \
0                42.374446      35.526797       51.085877       58.162041   
1                42.404536      45.731529       42.336170       44.246985   
2                44.682893      38.330116       36.968880       29.145712   
3                46.529622      41.824672       56.254175       46.868027   
4                30.453696      57.658592       24.447899       44.725130   

   Volatilidade_7  Retorno_Percentual  
0        3.710073            0.043834  
1        4.528431           -0

## 2. Carregamento do modelo e a geração das previsões


In [3]:
import joblib
import pandas as pd
import xgboost as xgb

# Carregar o modelo treinado
modelo_xgb = joblib.load("modelo_xgboost.pkl")

# Carregar a base aleatória gerada
df_nova = pd.read_csv("base_dados_aleatoria.csv")

# Remover a coluna de data para manter apenas as features do modelo
df_nova = df_nova.drop(columns=["Data_Negociacao"])

# Verificar se as colunas batem com as features do modelo treinado
features_treinadas = ["Preco_Abertura", "Maior_Preco_Dia", "Menor_Preco_Dia",
                       "Volume_Negociacoes_Proc", "Media_Movel_7", "Media_Movel_30",
                       "Media_Movel_90", "Volatilidade_7", "Retorno_Percentual"]

if list(df_nova.columns) == features_treinadas:
    print("✅ A base tem as mesmas colunas usadas no treinamento do modelo!")
else:
    print("⚠️ A base tem colunas diferentes do esperado. Verifique os nomes!")

dtest = xgb.DMatrix(df_nova)

# Fazer previsões com a nova base
previsoes = modelo_xgb.predict(dtest)

# Adicionar as previsões ao DataFrame
df_nova["Preco_Previsto"] = previsoes

# Exibir amostra dos dados previstos
print(df_nova.head())

# Salvar as previsões em um novo arquivo
df_nova.to_csv("previsoes_nova_base.csv", index=False)
print("✅ Previsões salvas no arquivo 'previsoes_nova_base.csv'")



✅ A base tem as mesmas colunas usadas no treinamento do modelo!
   Preco_Abertura  Maior_Preco_Dia  Menor_Preco_Dia  Volume_Negociacoes_Proc  \
0       34.981605        29.783770        28.642654                42.374446   
1       58.028572        53.529791        19.538941                42.404536   
2       49.279758        55.431402        51.987745                44.682893   
3       43.946339        47.451088        50.093574                46.529622   
4       26.240746        55.838687        25.317665                30.453696   

   Media_Movel_7  Media_Movel_30  Media_Movel_90  Volatilidade_7  \
0      35.526797       51.085877       58.162041        3.710073   
1      45.731529       42.336170       44.246985        4.528431   
2      38.330116       36.968880       29.145712        2.802548   
3      41.824672       56.254175       46.868027        2.894511   
4      57.658592       24.447899       44.725130        0.982274   

   Retorno_Percentual  Preco_Previsto  
0     

##  3. Importação da base de dados real


In [4]:
import pandas as pd

# Lendo o arquivo CSV (Base_Acoes_Uber.csv)
Base_Acoes_Uber = pd.read_csv(r'C:\Users\jmend\OneDrive\Documentos\Projetos\Uber Analysis\Base de Dados\uber_stock_data.csv')

# Um processo de validação na qual exibo os nomes das colunas do Data Frame para verificar se a importação do conjunto de dados deu certo.
Base_Acoes_Uber.columns

Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')

In [5]:
columns = ['Data_Negociacao', 'Preco_Ajustado_Fechamento', 'Preco_Fechamento', 'Maior_Preco_Dia', 'Menor_Preco_Dia',
           'Preco_Abertura', 'Volume_Negociacoes']

### --> Atribui os novos nomes de colunas ao conjunto de dados
Base_Acoes_Uber.columns = columns

### --> Capturo informações atualizadas dos dados
Base_Acoes_Uber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1444 entries, 0 to 1443
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Data_Negociacao            1444 non-null   object 
 1   Preco_Ajustado_Fechamento  1444 non-null   float64
 2   Preco_Fechamento           1444 non-null   float64
 3   Maior_Preco_Dia            1444 non-null   float64
 4   Menor_Preco_Dia            1444 non-null   float64
 5   Preco_Abertura             1444 non-null   float64
 6   Volume_Negociacoes         1444 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 79.1+ KB


In [6]:
import pandas as pd

# Gerar a sequência de datas para os próximos 90 dias
dias_futuros = 90
ultima_data = pd.to_datetime(Base_Acoes_Uber["Data_Negociacao"]).max()  # Última data da base original
datas_futuras = pd.date_range(start=ultima_data + pd.Timedelta(days=1), periods=dias_futuros, freq="D")

# Criar um novo DataFrame com as previsões
df_previsoes = pd.DataFrame(previsoes, columns=["Preco_Previsto"])

# Adicionar a coluna de datas
df_previsoes["Data_Negociacao"] = datas_futuras

# Reorganizar as colunas (opcional)
df_previsoes = df_previsoes[["Data_Negociacao", "Preco_Previsto"]]

# Salvar em CSV para análise
df_previsoes.to_csv("previsoes_com_datas.csv", index=False)

# Exibir amostra dos dados
print(df_previsoes.head())


  Data_Negociacao  Preco_Previsto
0      2025-02-06       29.948326
1      2025-02-07       37.367226
2      2025-02-08       52.912323
3      2025-02-09       50.232018
4      2025-02-10       40.139175
